# Folium : 지도 활용
- 세계지도를 기본 지원하고 다양한 스타일의 지도 이미지를 제공
- 쓸줄 알아야한다!

In [5]:
# Folium 설치하기
# ! pip install folium

In [6]:
import folium
import pandas as pd

In [7]:
# 서울 지도 만들기 (37.55, 126.98 <- 서울시청 위치)
seoul_map = folium.Map(location=[37.55, 126.98], zoom_start=12)
seoul_map

In [8]:
# 지도를 HTML 파일로 저장하기
seoul_map.save("../Data/seoul.html")

### 지도 스타일 적용
- tiles= 


In [9]:
seoul_map2 = folium.Map(
    location=[37.55, 126.98],
    tiles='Stamen Terrain',
    zoom_start=12
    )
seoul_map2

In [10]:
seoul_map3 = folium.Map(
    location=[37.55, 126.98],
    tiles='Stamen Toner',
    zoom_start=12
    )
seoul_map3

---
## 지도에 마커 표시하기
- 서울 시내 주요 대학교의 위치를 표시하기

In [11]:
df = pd.read_excel("../Data/서울지역 대학교 위치.xlsx")
df.columns = ['학교명','위도','경도']
df.head()

,학교명,위도,경도
0,KAIST 서울캠퍼스,37.592573,127.046737
1,KC대학교,37.548345,126.854797
2,가톨릭대학교(성신교정),37.585922,127.004328
3,가톨릭대학교(성의교정),37.499623,127.006065
4,감리교신학대학교,37.567645,126.961610


In [12]:
seoul_map = folium.Map(
    location=[37.55, 126.98],
    # tiles='Stamen Terrain',
    zoom_start=12
    )

# 대학교 위치 정보를 Marker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    #print(name, lat, len)
    #popup=> 클릭하면
    #add_to(seoul_map) 이 맵에 붙일꺼야
    folium.Marker([lat, lng], popup=name).add_to(seoul_map)

seoul_map

---
## 지도에 원형 마커 표시
- 데이터의 크기를 보고싶을때 사용
- 이것도 쓸줄 알아야함

In [13]:
seoul_map = folium.Map(
    location=[37.55, 126.98],
    # tiles='Stamen Terrain',
    zoom_start=12
    )

# 대학교 위치 정보를 CircleMarker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker(
        [lat, lng],
        radius= 10, # 원의 반지름
        color = 'magenta',
        fill = True,
        fill_color = 'blue',
        fill_opacity=0.7, # 투명도
        popup=name).add_to(seoul_map
        )


seoul_map

---
## 지도 영역에 단계구분포 (Choropleth Map) 표시하기
- 행정구역과 같이 지도상의 어떤 경계에 둘러싸인 영역에 색을 칠하거나 음영등의 정보를 나타내는 시각화 방법

In [14]:
import json

In [15]:
# 경기도 인구변화 데이터를 불러와서 데이터 프레임으로 변환
df = pd.read_excel("../Data/경기도인구데이터.xlsx", index_col='구분')
df.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
구분,,,,,,,,,,,
수원시장안구,287474,285803,290798,293692,290999,291757,300908,301196,299016,296479,293828
수원시권선구,310553,308177,304097,306783,321176,332633,331773,339835,351053,358393,365653
수원시팔달구,216945,213235,219833,216503,209772,206794,204805,203479,199180,198515,193311
수원시영통구,252730,260210,258421,260557,266542,289074,310671,329718,335375,340654,349836
성남시수정구,256744,248452,242161,241070,236123,233997,228167,223539,219531,232841,236932


In [16]:
# column 년도들이 숫자로 되어있어서 문자열로 바꿔줌
df.columns = df.columns.map(str)

In [17]:
# 경기도 시순수 겅계정보를 가진 geo-json 파일 불러오기
geo_path = '../Data/경기도행정구역경계.json'
try:
    geo_data = json.load(open(geo_path, encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path, encoding='utf-8-sig'))
#geo_data

In [18]:
# 경기도 지도 만들기
g_map = folium.Map(
    location=[37.5501, 126.982],
    tiles = 'Stamen Terrain',
    zoom_start=9
)

year = '2017'


# Choropleth 클래스로 단계구분 표시하기
folium.Choropleth(
    geo_data=geo_data, # 지도 경계
    data=df[year], # 표시하려는 데이터
    columns=[df.index, df[year]], # 열지정
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    threshold_scale = [10000, 100000, 300000, 500000, 700000], # 범위 지정하기
    key_on='feature.properties.name' # <- json 데이터 읽어서 가져온거야
).add_to(g_map)

g_map.save("../Data/gyonggi_population_" + year + ".html")
g_map